In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import logging
import time
import sys
from btfxwss import BtfxWss    
import requests
import urllib
from urllib.request import urlopen
from urllib import request
import json
import time
import hmac,hashlib
import datetime
%matplotlib inline

In [3]:
import krakenex

In [7]:
log = logging.getLogger(__name__)

fh = logging.FileHandler('test.log')
fh.setLevel(logging.DEBUG)
sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.DEBUG)

In [11]:
log.addHandler(sh)
log.addHandler(fh)
logging.basicConfig(level=logging.DEBUG, handlers=[fh, sh])
    

wss = BtfxWss()
wss.start()

INFO:btfxwss.connection:Connection opened
INFO:btfxwss.connection:API version: 2


In [13]:
while not wss.conn.connected.is_set():
    time.sleep(1)

# Subscribe to some channels
wss.subscribe_to_ticker('BTCUSD')
wss.subscribe_to_order_book('BTCUSD')



DEBUG:btfxwss.client:_subscribe: {'event': 'subscribe', 'channel': 'ticker', 'symbol': 'BTCUSD'}
DEBUG:btfxwss.client:_subscribe: {'event': 'subscribe', 'channel': 'book', 'symbol': 'BTCUSD'}
ERROR:btfxwss.connection:Already Subscribed
ERROR:btfxwss.connection:Already Subscribed


KeyboardInterrupt: 

In [18]:
# Do something else
ticker_q = wss.tickers('BTCUSD')
t = time.time()
while time.time() - t < 150:
    print(ticker_q.get())
    time.sleep(1)

([[17753, 67.01231961, 17754, 21.28104603, 1349, 0.0822, 17753, 59924.82407485, 18111, 16227]], 1513369497.4421093)
([[17754, 61.23030278, 17755, 16.62337984, 1351, 0.0824, 17755, 59925.32968955, 18111, 16227]], 1513369507.666992)
([[17756, 67.53683889, 17759, 16.7211448, 1355, 0.0826, 17759, 59923.01010714, 18111, 16227]], 1513369527.4538956)
([[17759, 66.33646782, 17763, 21.46295703, 1359, 0.0828, 17763, 59923.12018517, 18111, 16227]], 1513369537.4921842)
([[17772, 39.37099616, 17773, 17.93692895, 1376, 0.0839, 17780, 59928.17079357, 18111, 16227]], 1513369547.537036)
([[17774, 57.0342233, 17775, 31.73141865, 1371, 0.0836, 17775, 59928.06688178, 18111, 16227]], 1513369557.4316976)
([[17774, 55.21223172, 17775, 41.50995239, 1370, 0.0835, 17774, 59926.28990703, 18111, 16227]], 1513369567.6457853)
([[17774, 53.96277825, 17775, 40.52388898, 1371, 0.0836, 17775, 59921.30725716, 18111, 16227]], 1513369577.4812512)
([[17775, 48.69691263, 17777, 32.09697546, 1376, 0.0839, 17780, 59921.865963

In [10]:
# Unsubscribing from channels:
wss.unsubscribe_from_ticker('BTCUSD')
wss.unsubscribe_from_order_book('BTCUSD')

# Shutting down the client:
wss.stop()

INFO:btfxwss.connection:Connection closed


# Poloniex

In [49]:
 
def createTimeStamp(datestr, format="%Y-%m-%d %H:%M:%S"):
    return time.mktime(time.strptime(datestr, format))
 
class poloniex:
    def __init__(self, APIKey, Secret):
        self.APIKey = 'K91I1ELM-1N1WC8QW-Q6C3ZFEV-4HAY3V6E'
        self.Secret = '242eb8e2a4279f673e05e0e87c2905daf4b12a9ced6d43c50ffed7a4fa127788843cdc460697b875c048b1b9d40df567c9b206c1c4f250bf2d7b58e559b9de0e'
 
    def post_process(self, before):
        after = before
 
        # Add timestamps if there isnt one but is a datetime
        if('return' in after):
            if(isinstance(after['return'], list)):
                for x in xrange(0, len(after['return'])):
                    if(isinstance(after['return'][x], dict)):
                        if('datetime' in after['return'][x] and 'timestamp' not in after['return'][x]):
                            after['return'][x]['timestamp'] = float(createTimeStamp(after['return'][x]['datetime']))
                           
        return after
 
    def api_query(self, command, req={}):
 
        if(command == "returnTicker" or command == "return24Volume"):
            ret = urlopen('https://poloniex.com/public?command=' + command)
            return json.loads(ret.read())
        elif(command == "returnOrderBook"):
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + command + '&currencyPair=' + str(req['currencyPair'])))
            return json.loads(ret.read())
        elif(command == "returnMarketTradeHistory"):
            ret = urllib2.urlopen(urllib2.Request('https://poloniex.com/public?command=' + "returnTradeHistory" + '&currencyPair=' + str(req['currencyPair'])))
            return json.loads(ret.read())
        else:
            req['command'] = command
            req['nonce'] = int(time.time()*1000)
            post_data = urllib.urlencode(req)
 
            sign = hmac.new(self.Secret, post_data, hashlib.sha512).hexdigest()
            headers = {
                'Sign': sign,
                'Key': self.APIKey
            }
 
            ret = urlopen('https://poloniex.com/tradingApi', post_data, headers)
            jsonRet = json.loads(ret.read())
            return self.post_process(jsonRet)
 
 
    def returnTicker(self):
        return self.api_query("returnTicker")
 
    def return24Volume(self):
        return self.api_query("return24Volume")
 
    def returnOrderBook (self, currencyPair):
        return self.api_query("returnOrderBook", {'currencyPair': currencyPair})
 
    def returnMarketTradeHistory (self, currencyPair):
        return self.api_query("returnMarketTradeHistory", {'currencyPair': currencyPair})
 
 
    # Returns all of your balances.
    # Outputs:
    # {"BTC":"0.59098578","LTC":"3.31117268", ... }
    def returnBalances(self):
        return self.api_query('returnBalances')
 
    # Returns your open orders for a given market, specified by the "currencyPair" POST parameter, e.g. "BTC_XCP"
    # Inputs:
    # currencyPair  The currency pair e.g. "BTC_XCP"
    # Outputs:
    # orderNumber   The order number
    # type          sell or buy
    # rate          Price the order is selling or buying at
    # Amount        Quantity of order
    # total         Total value of order (price * quantity)
    def returnOpenOrders(self,currencyPair):
        return self.api_query('returnOpenOrders',{"currencyPair":currencyPair})
 
 
    # Returns your trade history for a given market, specified by the "currencyPair" POST parameter
    # Inputs:
    # currencyPair  The currency pair e.g. "BTC_XCP"
    # Outputs:
    # date          Date in the form: "2014-02-19 03:44:59"
    # rate          Price the order is selling or buying at
    # amount        Quantity of order
    # total         Total value of order (price * quantity)
    # type          sell or buy
    def returnTradeHistory(self,currencyPair):
        return self.api_query('returnTradeHistory',{"currencyPair":currencyPair})
 
    # Places a buy order in a given market. Required POST parameters are "currencyPair", "rate", and "amount". If successful, the method will return the order number.
    # Inputs:
    # currencyPair  The curreny pair
    # rate          price the order is buying at
    # amount        Amount of coins to buy
    # Outputs:
    # orderNumber   The order number
    def buy(self,currencyPair,rate,amount):
        return self.api_query('buy',{"currencyPair":currencyPair,"rate":rate,"amount":amount})
 
    # Places a sell order in a given market. Required POST parameters are "currencyPair", "rate", and "amount". If successful, the method will return the order number.
    # Inputs:
    # currencyPair  The curreny pair
    # rate          price the order is selling at
    # amount        Amount of coins to sell
    # Outputs:
    # orderNumber   The order number
    def sell(self,currencyPair,rate,amount):
        return self.api_query('sell',{"currencyPair":currencyPair,"rate":rate,"amount":amount})
 
    # Cancels an order you have placed in a given market. Required POST parameters are "currencyPair" and "orderNumber".
    # Inputs:
    # currencyPair  The curreny pair
    # orderNumber   The order number to cancel
    # Outputs:
    # succes        1 or 0
    def cancel(self,currencyPair,orderNumber):
        return self.api_query('cancelOrder',{"currencyPair":currencyPair,"orderNumber":orderNumber})
 
    # Immediately places a withdrawal for a given currency, with no email confirmation. In order to use this method, the withdrawal privilege must be enabled for your API key. Required POST parameters are "currency", "amount", and "address". Sample output: {"response":"Withdrew 2398 NXT."}
    # Inputs:
    # currency      The currency to withdraw
    # amount        The amount of this coin to withdraw
    # address       The withdrawal address
    # Outputs:
    # response      Text containing message about the withdrawal
    def withdraw(self, currency, amount, address):
        return self.api_query('withdraw',{"currency":currency, "amount":amount, "address":address})

In [50]:
poloniex.returnTicker('BTC_USD')

AttributeError: 'str' object has no attribute 'api_query'

In [51]:
command = 'returnTicker'
a = urlopen('https://poloniex.com/public?command=' + 'returnTicker')
a.read()

b'{"BTC_BCN":{"id":7,"last":"0.00000038","lowestAsk":"0.00000039","highestBid":"0.00000038","percentChange":"0.11764705","baseVolume":"322.33075250","quoteVolume":"952592498.90793753","isFrozen":"0","high24hr":"0.00000039","low24hr":"0.00000027"},"BTC_BELA":{"id":8,"last":"0.00002051","lowestAsk":"0.00002052","highestBid":"0.00002050","percentChange":"0.25213675","baseVolume":"65.71540650","quoteVolume":"3336841.68996731","isFrozen":"0","high24hr":"0.00002266","low24hr":"0.00001578"},"BTC_BLK":{"id":10,"last":"0.00003500","lowestAsk":"0.00003551","highestBid":"0.00003500","percentChange":"0.09649122","baseVolume":"16.34979790","quoteVolume":"509338.70753161","isFrozen":"0","high24hr":"0.00003601","low24hr":"0.00002823"},"BTC_BTCD":{"id":12,"last":"0.02300001","lowestAsk":"0.02300711","highestBid":"0.02300001","percentChange":"0.56711686","baseVolume":"314.66396478","quoteVolume":"13856.20414172","isFrozen":"0","high24hr":"0.02850000","low24hr":"0.01426288"},"BTC_BTM":{"id":13,"last":"0

# Kraken

In [11]:
import krakenex
#https://python3-krakenex.readthedocs.io/en/latest/
#https://www.kraken.com/help/api#get-asset-info

k = krakenex.API(key='en0i96UpL6N62X23/hxVgAAfNIZMWhp78LFMc3441Yor4NeCjSIV/OhM',secret='LnLXo4FQqsrhNqubPU9F7VBifC75dNa4yC+QjMjXhQbYoc3Sr7BgFfBYLVsFyWgn7kLj6Xk9c/vSiVtT/hx/YA==')

In [12]:
k

### Разница

In [14]:
data_spread = k.query_public('Spread',{'pair': 'XBTUSD'})
spread_time = []
spread_bid = []
spread_ask = []

for i in range(0, len(data_spread['result']['XXBTZUSD'])):
    spread_time.append(datetime.datetime.fromtimestamp(data_spread['result']['XXBTZUSD'][i][0]).strftime('%Y-%m-%d %H:%M:%S'))
    spread_bid.append(data_spread['result']['XXBTZUSD'][i][1])
    spread_ask.append(data_spread['result']['XXBTZUSD'][i][2])

spread_dict = {'spread_time':spread_time,'spread_bid':spread_bid,'spread_ask':spread_ask}

spread_df = pd.DataFrame(spread_dict)
spread_df.head(5)

,spread_ask,spread_bid,spread_time
0,15024.60000,15011.20000,2017-12-26 12:10:12
1,15024.40000,15011.20000,2017-12-26 12:10:14
2,15024.40000,15011.50000,2017-12-26 12:10:20
3,15024.40000,15011.60000,2017-12-26 12:10:21
4,15032.80000,15011.60000,2017-12-26 12:10:28


In [16]:
data_spread = k.query_public('Spread',{'pair': 'XBTUSD'})
spread_time = []
spread_bid = []
spread_ask = []

for i in range(0, len(data_spread['result']['XXBTZUSD'])):
    spread_time.append(datetime.datetime.fromtimestamp(data_spread['result']['XXBTZUSD'][i][0]).strftime('%Y-%m-%d %H:%M:%S'))
    spread_bid.append(data_spread['result']['XXBTZUSD'][i][1])
    spread_ask.append(data_spread['result']['XXBTZUSD'][i][2])

spread_dict = {'spread_time':spread_time,'spread_bid':spread_bid,'spread_ask':spread_ask}

spread_df = pd.DataFrame(spread_dict)
spread_df

,spread_ask,spread_bid,spread_time
0,15037.90000,15011.60000,2017-12-26 12:10:28
1,15025.80000,15011.60000,2017-12-26 12:10:30
2,15025.80000,15014.60000,2017-12-26 12:10:30
3,15025.80000,15011.60000,2017-12-26 12:10:33
4,15037.00000,15011.60000,2017-12-26 12:10:43
5,15025.60000,15011.60000,2017-12-26 12:10:45
6,15024.40000,15011.60000,2017-12-26 12:11:01
7,15024.40000,15010.30000,2017-12-26 12:11:03
8,15024.40000,15024.40000,2017-12-26 12:11:07
9,15024.40000,15010.30000,2017-12-26 12:11:08


### Стоимость

In [157]:
data_OHLC = k.query_public('OHLC',{'pair': 'XBTUSD','interval':5})
ohlc_time = []
ohlc_open=[]
ohlc_close=[]
ohlc_high=[]
ohlc_low=[]
ohlc_volume=[]
ohlc_price = []
for i in range(0,len(data_OHLC['result']['XXBTZUSD'])):
    ohlc_time.append(datetime.datetime.fromtimestamp(data_OHLC['result']['XXBTZUSD'][i][0]).strftime('%Y-%m-%d %H:%M:%S'))
    ohlc_open.append(data_OHLC['result']['XXBTZUSD'][i][1])
    ohlc_high.append(data_OHLC['result']['XXBTZUSD'][i][2])
    ohlc_low.append(data_OHLC['result']['XXBTZUSD'][i][3])
    ohlc_close.append(data_OHLC['result']['XXBTZUSD'][i][4])
    ohlc_price.append(data_OHLC['result']['XXBTZUSD'][i][5])
    ohlc_volume.append(data_OHLC['result']['XXBTZUSD'][i][6])

ohlc_dict = {'ohlc_time':ohlc_time,'ohlc_price':ohlc_price,'ohlc_open':ohlc_open,
             'ohlc_close':ohlc_close,'ohlc_low':ohlc_low,'ohlc_high':ohlc_high,'ohlc_volume':ohlc_volume}
ohlc_df = pd.DataFrame(ohlc_dict)
ohlc_df.head(5)
    
    

DEBUG:urllib3.connectionpool:https://api.kraken.com:443 "POST /0/public/OHLC HTTP/1.1" 200 None


,ohlc_close,ohlc_high,ohlc_low,ohlc_open,ohlc_price,ohlc_time,ohlc_volume
0,16206.4,16206.4,16010.4,16131.9,16125.1,2017-12-21 03:25:00,4.34002466
1,16298.4,16350.0,16159.0,16159.0,16243.1,2017-12-21 03:30:00,14.82279158
2,16350.0,16390.0,16297.1,16297.1,16370.2,2017-12-21 03:35:00,8.64257878
3,16310.0,16377.0,16307.0,16360.5,16326.3,2017-12-21 03:40:00,16.23585777
4,16307.0,16360.0,16307.0,16307.1,16307.8,2017-12-21 03:45:00,16.90389976


In [37]:
import krakenex

ModuleNotFoundError: No module named 'krakenex'

### Стакан

In [180]:
data_depth = k.query_public('Depth',{'pair': 'XBTUSD','count':100})
ask_price = []
ask_volume = []
ask_time = []
for i in range(0,len(data_depth['result']['XXBTZUSD']['asks'])):
    ask_price.append(data_depth['result']['XXBTZUSD']['asks'][0:100][i][0])
    ask_volume.append(data_depth['result']['XXBTZUSD']['asks'][0:100][i][1])
    ask_time.append(datetime.datetime.fromtimestamp(data_depth['result']['XXBTZUSD']['asks'][0:100][i][2]).strftime('%Y-%m-%d %H:%M:%S'))

bid_price = []
bid_volume = []
bid_time = []
for i in range(0,len(data_depth['result']['XXBTZUSD']['bids'])):
    bid_price.append(data_depth['result']['XXBTZUSD']['bids'][0:100][i][0])
    bid_volume.append(data_depth['result']['XXBTZUSD']['bids'][0:100][i][1])
    bid_time.append(datetime.datetime.fromtimestamp(data_depth['result']['XXBTZUSD']['bids'][0:100][i][2]).strftime('%Y-%m-%d %H:%M:%S'))

depth_dict = {'ask_price':ask_price,'ask_volume':ask_volume,'ask_time':ask_time,'bid_price':bid_price,'bid_volume':bid_volume,'bid_time':bid_time}
depth_df = pd.DataFrame(depth_dict)
depth_df.head(5)

DEBUG:urllib3.connectionpool:https://api.kraken.com:443 "POST /0/public/Depth HTTP/1.1" 200 None


,ask_price,ask_time,ask_volume,bid_price,bid_time,bid_volume
0,15296.10000,2017-12-23 17:29:35,0.063,15296.10000,2017-12-23 17:29:29,0.344
1,15299.40000,2017-12-23 17:29:35,0.118,15296.00000,2017-12-23 17:24:39,18.676
2,15308.00000,2017-12-23 17:29:27,0.059,15291.00000,2017-12-23 17:24:43,0.891
3,15353.00000,2017-12-23 17:27:27,0.027,15290.00000,2017-12-23 17:27:28,0.013
4,15359.00000,2017-12-23 15:36:23,0.050,15265.20000,2017-12-23 17:22:43,0.400


INFO:btfxwss.connection:Connection Error - [WinError 10054] An existing connection was forcibly closed by the remote host
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection opened
INFO:btfxwss.connection:_on_open(): Connection reconnected, re-subscribing..
INFO:btfxwss.connection:API version: 2
INFO:btfxwss.queue_processor:Subscription succesful for channel ('ticker', 'BTCUSD')
INFO:btfxwss.queue_processor:Subscription succesful for channel ('book', 'BTCUSD')


# Bitfinex

In [171]:
import logging
import time
import sys
from btfxwss import BtfxWss

log = logging.getLogger(__name__)

fh = logging.FileHandler('test.log')
fh.setLevel(logging.DEBUG)
sh = logging.StreamHandler(sys.stdout)
sh.setLevel(logging.DEBUG)

log.addHandler(sh)
log.addHandler(fh)
logging.basicConfig(level=logging.DEBUG, handlers=[fh, sh])
    

wss = BtfxWss()
wss.start()

INFO:btfxwss.connection:Connection opened
INFO:btfxwss.connection:API version: 2
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.


In [173]:
wss.subscribe_to_ticker('BTCUSD')
wss.subscribe_to_order_book('BTCUSD')
wss.stop()

ERROR:btfxwss.client:Cannot call subscribe_to_ticker() on unestablished connection!
ERROR:btfxwss.client:Cannot call subscribe_to_order_book() on unestablished connection!
INFO:btfxwss.connection:Connection opened
INFO:btfxwss.connection:API version: 2
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - EOF occurred in violation of protocol (_ssl.c:777)
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection opened
INFO:btfxwss.connection:_on_open(): Connection reconnected, re-subscribing..
INFO:btfxwss.connection:API version: 2
INFO:btfxwss.queue_processor:Subscription succesfu

# Bittrex

In [ ]:
api = 'https://bittrex.com/api/v1.1/public/'
requests.get(api+'')

In [176]:
import bitstampy
from bitstampy import api
api.ticker()
api.ticker('BTC')

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): www.bitstamp.net


ConnectionError: HTTPSConnectionPool(host='www.bitstamp.net', port=443): Max retries exceeded with url: /api/ticker/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x00000188D5733EB8>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',))

INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attempting to connect again in 10 seconds.
INFO:btfxwss.connection:Connection Error - [Errno 11001] getaddrinfo failed
INFO:btfxwss.connection:Connection closed
INFO:btfxwss.connection:Attemptin